### --- Day 9: Disk Fragmenter ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/9

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 9);
WriteLines(inputLines, maxCols: 80);

Loading puzzle file: Day9.txt
Total lines: 1
Max line length: 19999

37177921644951938277999269384375694569563018897314838836721412555971912060537033


In [4]:
var inputLine = inputLines[0];

In [5]:
var testInputLine = "2333133121414131402";

Whoa, that puzzle input looks like it could render into an impossibly huge buffer! Let's see exactly how big it would be...

In [6]:
int CalcBufferSize(string inputLine) => inputLine.ToCharArray().Select(CharToDigit).Sum();

Console.WriteLine(CalcBufferSize(testInputLine));
Console.WriteLine(CalcBufferSize(inputLine));

42
95186


Ok, 95K is not too bad. I think the simplest approach will be to maintain pointers at each end of a buffer, and move items from the end to the start until the pointers cross over.

In [7]:
long DefragAndChecksum(string inputLine)
{
    var buffer = CreateBuffer(inputLine);
    Defrag(buffer);
    return CalculateChecksum(buffer);
}

// Use 0 to represent empty. This creates a slight problem as the file IDs also
// start at 0. But we'll mitigate that by starting the file IDs at 1 and correcting
// them during the checksum
const int EMPTY = 0;

int[] CreateBuffer(string inputLine)
{
    var bufferSize = CalcBufferSize(inputLine);
    var buffer = new int[bufferSize];

    var inputInts = inputLine.ToCharArray().Select(CharToDigit);

    var fileId = 1;
    var pointer = 0;

    void writeFile(int length)
    {
        foreach (var i in Enumerable.Range(0, length))
        {
            buffer[pointer++] = fileId;
        }
        fileId++;
    }

    foreach (var (index, length) in inputInts.Index())
    {
        if (index % 2 == 0)
        {
            writeFile(length);
        }
        else
        {
            // Empty space, already 0
            pointer += length;
        }
    }

    return buffer;
}

void Defrag(int[] fileBuffer)
{
    int rightPointer = fileBuffer.Length - 1;
    int leftPointer = 0;

    SafetyLimit safetyLimit = new();

    while (true)
    {
        safetyLimit.EnsureBelow(50_000);

        while (fileBuffer[rightPointer] == EMPTY)
        {
            rightPointer--;
        }
        while (fileBuffer[leftPointer] != EMPTY)
        {
            leftPointer++;
        }

        if (leftPointer >= rightPointer)
        {
            return;
        }

        // Swap
        fileBuffer[leftPointer] = fileBuffer[rightPointer];
        fileBuffer[rightPointer] = EMPTY;
    }
}

long CalculateChecksum(int[] fileBuffer) => fileBuffer.Where(i => i != EMPTY).Select((item, i) => i * (long)(item - 1)).Sum();

In [8]:
// ...In this example, the checksum is the sum of these, 1928.

var testAnswer = DefragAndChecksum(testInputLine);
Console.WriteLine(testAnswer);

1928


In [9]:
var part1Answer = DefragAndChecksum(inputLine);
Console.WriteLine(part1Answer);

6421128769094


In [10]:
// 6421128769094 is correct!
Ensure(6421128769094, part1Answer);